# Creation table Olivier_it

`IT_Equipment_v2` : Table IT_Equipment provenant d'un fichier csv

DROP TABLE IF EXISTS `IT_Equipment_v2` ;
CREATE TABLE IT_Equipment_v2
as
SELECT * FROM `IT_Equipment`
where nom_de_la_salle in ('P1', 'P2', 'P3', 'P4');

`assets_filtree_v2` : Table assets_new (CMDB) filtré aux deux localisations : FRVLB0C2 / FRTCRAVA

DROP TABLE IF EXISTS `assets_filtree_v2` ;
CREATE TABLE assets_filtree_v2
as
select *
from `assets_new`
where (API = 'FRVLB0C2' or API = 'FRTCRAVA') ;

ALTER TABLE assets_filtree_v2
ADD COLUMN Rack varchar(255),
ADD COLUMN Slot varchar(255),
ADD COLUMN Slot_begin varchar(255),
ADD COLUMN Slot_end varchar(255);

UPDATE assets_filtree_v2
SET 
	Rack = SUBSTRING_INDEX(DALLE, '/', 1)  ,
	Slot = SUBSTRING_INDEX(DALLE, '/', -1) ,
	Slot_begin = SUBSTRING_INDEX(SUBSTRING_INDEX(DALLE, '/', -1), '-', 1) ,
	Slot_end = SUBSTRING_INDEX(DALLE, '-', -1)

DROP TABLE IF EXISTS `IT_Equipment_olivier_no_match` ;
CREATE TABLE IT_Equipment_olivier_no_match
as
SELECT it.*
FROM `IT_Equipment_v2` it
LEFT JOIN `olivier_it` oit on oit.olivier_index = CONCAT(it.Nom_de_la_salle, it.reperage_id)

where it.nom_de_la_salle in ('P1', 'P2', 'P3', 'P4') and oit.olivier_index is null
ORDER BY Reperage_ID asc

DROP TABLE IF EXISTS `assets_filtree_reduced` ;
CREATE TABLE assets_filtree_reduced
AS
SELECT ype, subtype, ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, No_serie, etat, api, tag_partenaire
from `assets_filtree_v2` aa
WHERE SALLE in ('P1', 'P2', 'P3', 'P4', '0')

ALTER TABLE `assets_filtree_reduced`
ADD COLUMN reperage_id varchar(255) ;

update `assets_filtree_reduced`
set reperage_id = concat(Rack,":Slot ", slot_begin)


`IT_Equipment_match_position_reduced_v3` : groupement des lignes par positions et affichage d'un seul asset_id par positions + nombres d'equipements sur la même position

In [ ]:
DROP TABLE IF EXISTS IT_Equipment_match_position_reduced_v3; 
CREATE TABLE IT_Equipment_match_position_reduced_v3
as
SELECT it.nom_de_la_salle, it.reperage_id, count(it.reperage_id) as RowNumber,
		concat(aa.asset_id) asset_id

FROM   `olivier_it` it  
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.reperage_id = aa.reperage_id
)  
GROUP BY it.reperage_id

-----

In [ ]:
ALTER TABLE olivier_it
ADD COLUMN STATUS_2 varchar(255),
ADD COLUMN Nom_de_baie varchar(255),
ADD COLUMN Slot_u varchar(255),
ADD COLUMN STATUS_3 varchar(255),
ADD COLUMN ASSET_ID_OK_POSITION varchar(255),
ADD COLUMN status varchar(255) ;

### `Colonnes de status differentes`

In [ ]:
ALTER TABLE olivier_it 
ADD COLUMN status_asset_id varchar(255),
ADD COLUMN status_position varchar(255),
ADD COLUMN status_name varchar(255),
ADD COLUMN status_etat varchar(255),
ADD COLUMN status_global varchar(255),
ADD COLUMN status_etat_hpe varchar(255),
ADD COLUMN status_spec varchar(255),
ADD COLUMN status_cc varchar(255) ;

In [ ]:
UPDATE `olivier_it`
set
Nom_de_baie = SUBSTRING_INDEX(reperage_id,':',1) ,
Slot_u = SUBSTRING_INDEX(reperage_id,'Slot',-1) ;

# UPDATE

## `Status` 

attribut `Status` :

    - Commun : Equipment present sur fichier Olivier_it & It_equipment
    - Non trouvé : Equipment non present sur It_equipment

In [ ]:
UPDATE `olivier_it` oit
INNER JOIN `IT_Equipment_v2` it on oit.olivier_index = CONCAT(it.Nom_de_la_salle, it.reperage_id)

SET oit.status = 'Commun'

where it.nom_de_la_salle in ('P1', 'P2', 'P3', 'P4')

In [ ]:
UPDATE `olivier_it` oit
LEFT JOIN `IT_Equipment_v2` it on oit.olivier_index = CONCAT(it.Nom_de_la_salle, it.reperage_id)

set oit.status = 'Non trouvé'

where it.reperage_id is null

### `MATCH ASSET_ID`

attribut `status_asset_id` : 

    - "Trouvé - bon asset id"
    - "Non Trouvé - Mauvais Asset ID" impossibilité de trouver l'asset_id dans la CMDB (assets_filtrée)

In [ ]:
UPDATE `olivier_it` it
LEFT JOIN `assets_filtree_reduced` aa
ON aa.asset_id = it.asset_id
and it.asset_id != '0'
SET it.status_asset_id = ( CASE WHEN aa.asset_id is not null then 'Trouvé - bon asset id'
                          ELSE "Non Trouvé - Mauvais Asset ID"
                          END 
                          )


### `MATCH POSITION`

attribut `status_position` : 

    - "Bonne localisation"
    - "KO localisation" : impossibilité de trouver la position dans la CMDB (assets_filtrée)

In [ ]:

UPDATE `olivier_it` it 
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
    
INNER JOIN `IT_Equipment_match_position_reduced_v3` it2
ON it.reperage_id = it2.reperage_id
and it.nom_de_la_salle = it2.nom_de_la_salle

SET it.status_position = 'Bonne localisation'
WHERE  (
   aa.model not like '%BL460C%'
OR aa.model not like '%BL660C%'
OR aa.model not like '%BL20P%'
OR aa.model not like '%BL860C%'
OR aa.model not like '%BL420C%'
OR aa.model not like '%BL680C%'
OR aa.model not like '%WS460C%'
OR aa.model not like '%BL870C%' )

In [ ]:
UPDATE `olivier_it` it 
SET it.status_position = 'KO localisation'
where it.status_position is null

# `Nom`

attribut `status_nom_court` : Priorisation du status affiché 
    
    1. Présence d'asset d'id
        1. "nom_court type OLD"
        2. "nom_long type OLD" 
        3. "OK nom_court"
        4. "OK nom_long"
        5. "Mauvais nom" : nom court/long non présent dans la cmdb
    
    2. Non présence d'asset d'id
        1. "nom_court type OLD"
        2. "nom_long type OLD" 
        3. "OK nom_court"
        4. "OK nom_long"
        5. "Mauvais nom" : nom court/long non présent dans la cmdb

In [ ]:
UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON aa.nom_court  = it.short_name    
SET it.status_name = "OK nom_court"
where it.short_name != ''

In [ ]:
UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON aa.nom_long  = it.nom   
SET it.status_name = "OK nom_long"
where it.nom != '' and status_name is null

### `'CMDB-OK-OLD'`

In [ ]:
UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name )
AND aa.asset_id = it.asset_id


SET it.status_name = (CASE WHEN nom_court like '%OLD' and  nom !='' THEN "nom_court type OLD"
                      WHEN nom_long like '%OLD' and  short_name !='' THEN "nom_long type OLD"
                      ELSE it.status_name
                      END
                     )

where (aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD')

In [ ]:
UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name


SET it.status_name = (CASE WHEN nom_court like '%OLD' and  nom !='' THEN "nom_court type OLD"
                      WHEN nom_long like '%OLD' and  short_name !='' THEN "nom_long type OLD"
                      ELSE it.status_name
                      END
                     )

where (aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD') and status_asset_id != 'Trouvé - bon asset id'

In [ ]:
UPDATE `olivier_it` it
SET it.status_name = "Mauvais nom"
where it.status_name is null

# `Status : Etat`

attribut `status_etat` : Si un équipment est repéré (par son asset_id et/ou sa position ) 

    - "Trouvé - bon asset id"
    - "Non Trouvé - Mauvais Asset ID"
    - "KO Etat" - Aucun ou plusieurs equipments sur la même position, impossible d'établir le lien

In [ ]:
UPDATE `olivier_it` it
LEFT JOIN `assets_filtree_reduced` aa
ON aa.asset_id = it.asset_id
and it.asset_id != '0'
SET it.status_etat = aa.etat ;


In [ ]:

UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name )

SET it.status_etat = aa.etat
where status_name = "nom_court type OLD" ;

In [ ]:

UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name )
SET it.status_etat = aa.etat
where status_name = "nom_long type OLD" ;

In [ ]:

UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name )
AND aa.asset_id = it.asset_id

SET it.status_etat = aa.etat
where status_name = "nom_court type OLD" and status_asset_id != 'Trouvé - bon asset id'

In [ ]:

UPDATE `olivier_it` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name )
AND aa.asset_id = it.asset_id
SET it.status_etat = aa.etat
where status_name = "nom_long type OLD" and status_asset_id != 'Trouvé - bon asset id'

In [ ]:
UPDATE `olivier_it` it
 SET it.status_etat = 'KO Etat_CMDB'
where it.status_etat is null

# `Status_spec : Cas spécifique`

attribut `status_spec` : 

    - "Patch Panel"
    - "PCP"

### `Patch Panel`

In [ ]:
UPDATE `olivier_it` it 
SET it.Status_spec = "Trouvé - Patch Panel"
where type like '%Patch Panel%' or fabricant in ('Arista', 'Alcatel-Lucent');

### `PCP`

In [ ]:

UPDATE `olivier_it` it
SET it.Status_spec = 'Trouvé - PCP Mal documenté'
where (it.Nom like '%compute%' or it.short_name like '%compute%' or reperage_id like '%compute%') or 
(it.Nom like '%_Rack%' or it.short_name like '%_Rack%' or reperage_id like '%_Rack%') and (asset_id is null or asset_id ='0' or asset_id ='');

# `Status : Etat_hpe`

attribut `status_etat_hpe` : 

    1- pas asset id & pas de localisation
    2- pas asset id & localisation
    3- assed id & localisation => status
    4- asset id & pas localisation ==> "assetid"
    5- PCP ==> "PCP"
    6- Patch Panel => "Patch Panel"

In [ ]:
UPDATE `olivier_it` it
SET it.status_etat_hpe = 
    (CASE WHEN status_asset_id != 'Trouvé - bon asset id' AND status_position = 'KO localisation' and status_spec is null THEN 'pas asset id & pas de localisation'
     WHEN status_asset_id != 'Trouvé - bon asset id' AND status_position != 'KO localisation' and status_spec is null THEN 'pas asset id & localisation'
     WHEN status_asset_id = 'Trouvé - bon asset id' AND status_position != 'KO localisation' and status_spec is null THEN status_etat
     WHEN status_asset_id = 'Trouvé - bon asset id' AND status_position = 'KO localisation' and status_spec is null THEN 'asset id & pas de localisation'
     WHEN status_spec is not null then status_spec
     END
    )              
    ;

# `GLOBAL STATUS`

attribut `status_status` : Agrégation de tous les attribus

In [ ]:
UPDATE olivier_it 
set status_global = concat(status_asset_id , ' - ' , status_position , ' - ' , status_name, " - " , status_etat );

# `STATUS_CC`

attribut `status_CC` : Aggrégation de tous les attribus

In [ ]:
UPDATE olivier_it 
set status_cc = concat("6Sigma Hpe ", status_hpe , ' - ' ,
                       "6Sigma C2 " , status,  ' - ' ,
                       "CHIPRE ", status_etat_hpe ) ;


# Indicateur

In [ ]:
drop VIEW if EXISTS indicator_olivier_it_status ;
create VIEW indicator_olivier_it_status
as 
SELECT status_cc, count(*) row_count FROM `olivier_it`
GROUP BY status_cc

In [ ]:
drop view if EXISTS indicator_olivier_it_add_row_per_salle ;
create view indicator_olivier_it_add_row_per_salle
as 
select DISTINCT(from_source), NOM_de_la_salle, count(*) equipment_count from `olivier_it`
GROUP BY from_source, NOM_de_la_salle

## Controle, pour référence

In [ ]:
drop view if EXISTS `control_vision_spatial` ;
create view `control_vision_spatial` 
as
select * from `olivier_it`
where asset_id != '' and asset_id != '0' and asset_id is not null and status_cmdb = 'OK' and status_6sigma = 'OK' and status_etat != 'To be deleted'
group by FROM_source

### MAJ 30/08/2022

drop table if EXISTS olivier_it_copy_copy ;# MySQL returned an empty result set (i.e. zero rows).
create table olivier_it_copy_copy	
as
select * from `olivier_it_copy`# 2799 rows affected.

select * from `olivier_it_copy_copy`where olivier_index = 'P3AC11:Slot 15'

delete from `olivier_it_copy_copy`where olivier_index = 'P3AC11:Slot 15'

select it.* from `olivier_it` it
LEFT JOIN `olivier_it_copy_copy` it2 ON it.olivier_index = it2.olivier_index and it.nom = it2.nom
where it2.olivier_index is null
order by olivier_index desc

`olivier_it_records` : historique de tous les equipemnts UNIQUES qui sont passés par les fichiers CSV Olivier_it. Permet de checker si un equipements a été supprimer ou non par comparaison avec la dernière version du fichier CSV Olivier_it

CREATE TABLE olivier_it_records
as
select *, NULL status_deleted from `olivier_it`

select it2.modele, it2.nom, it2.olivier_index from `olivier_it` it
RIGHT JOIN `olivier_it_records` it2 ON it.olivier_index = it2.olivier_index and it.nom = it2.nom
where it.olivier_index is null

#### `status_deleted` : si l'equipement n'est pas présent sur la dernière version de olivier_it mais qu'elle est présente dans la table olivier_it_records, l'equipement est jugé comme delete

In [ ]:
drop VIEW if EXISTS olivier_it_deleted_records ;
create view olivier_it_deleted_records 
as
select it2.* from `olivier_it` it
RIGHT JOIN `olivier_it_records` it2 ON it.olivier_index = it2.olivier_index and it.nom = it2.nom
where it.olivier_index is null

In [ ]:
drop VIEW if EXISTS olivier_it_added_records ;
create view olivier_it_added_records 
as
select it.* from `olivier_it` it
LEFT JOIN `olivier_it_records` it2 ON it.olivier_index = it2.olivier_index and it.nom = it2.nom
where it2.olivier_index is null

In [ ]:
ALTER TABLE olivier_it
ADD COLUMN status_deleted varchar(255)

ajout des equipements supprimés `olivier_it`

In [ ]:
INSERT INTO `olivier_it`
SELECT *, 'DELETED' from `olivier_it_deleted_records`

ajout des nouveaux equipements à `olivier_it_records`

In [ ]:
INSERT INTO `olivier_it_records`
SELECT *, 'ADDED' from `olivier_it_added_records`

In [ ]:
select * from `olivier_it_records` 

where olivier_index = 'P2AB27:Slot 01'

Beaucoup d'equipment sont suppr de olivier it puis remis sur le records à l'aide de la CMDB/6SIGMA

In [ ]:
select olivier_index, group_concat(from_source) from_source, count(DISTINCT(ASSET_ID)) asset_count, count(DISTINCT(status_deleted)) count_stat, count(*)  from `olivier_it`
GROUP BY olivier_index
ORDER by count(DISTINCT(status_deleted)) desc


Nexus 2248TP GE Fabric Extender (Port Side Airflow Intake);Switch;Cisco;AA_SWC2P430_FEX112;;;;;;P4;W10:Slot 38;W10:Slot 38;;6sigma Hpe;;;P4W10:Slot 38;0-EXISTANT P3/P4;Cecile Pialot;;;;P4W10:Slot 38;Commun;Non Trouvé - Mauvais Asset ID - KO localisation - Mauvais nom - KO Etat_CMDB;6Sigma Hpe 6sigma Hpe - 6Sigma C2 Commun - CHIPRE pas asset id & pas de localisation;;Non Trouvé - Mauvais Asset ID;KO localisation;Mauvais nom;KO Etat_CMDB;;;;